In [1]:
#r "nuget:Serilog.Formatting.Compact.Reader,1.0.3"
#r "nuget:System.Linq.Async,4.0.0"

Installed package Serilog.Formatting.Compact.Reader version 1.0.3

Installed package System.Linq.Async version 4.0.0

In [1]:
using System.IO;
using Serilog.Events;
using Serilog.Formatting.Compact.Reader;
using System.Collections.Generic;
using System.Linq;

In [1]:
public async IAsyncEnumerable<(DateTimeOffset x, float y)> GetBestFrameCount()
{
    var compactFile = new FileInfo(@"C:\Users\vaido\SourceCode\MrBabyData\FrameRateAnalysis\RogueTD_PerformanceMetricsFrameRates.clef");
    using(var stream = compactFile.OpenText())
    {
        while(stream.Peek() > 0){
            var logevent = Serilog.Formatting.Compact.Reader.LogEventReader.ReadFromString(await stream.ReadLineAsync());
            yield return (
                logevent.Timestamp,
                float.Parse(logevent.Properties["bestDuration"].ToString())
            );
        }
    }
}

public struct PlotDatum{
    public double x {get; set;}
    public double y {get; set;}
}

var readData = await GetBestFrameCount().ToListAsync();
var firstTime = readData.First().x;
var data = readData.Select(i => new PlotDatum{x = (i.x - firstTime).TotalSeconds, y = 60/i.y}).ToList();
data

index,x,y
0,0,20849.259765625
1,1.0020632,20558.505859375
2,2.0043952,19805.90234375
3,3.005243,18953.154296875
4,4.0053185,17962.458984375
5,5.007538,17403.412109375
6,6.0112836,17095.96484375
7,7.0137795,16350.111328125
8,8.0143054,15751.751953125
9,9.0174597,15130.8818359375


In [1]:
#!javascript
var data;
interactive.csharp.getVariable("plotDatums")
    .then(result => data = result);


In [1]:
console.log(data)

Error: (1,1): error CS0103: The name 'console' does not exist in the current context

In [1]:
#!javascript
notebookScope.plot = (sgvSelector, dataVariable) => {
    let dtreeLoader = interactive.configureRequire({
        paths: {
            d3: "https://d3js.org/d3.v6.min"
        }
    });

    dtreeLoader(["d3"], function (d3) {
    var fakeData = [
        {
          x: 0,
          y : 0
        },
        {
          x : 20,
          y : 50
        },
        {
          x : 100,
          y : 15
        }
      ]

    var buildFrameRatePlot = (svg, width, height, data) => {
        var layoutMeasures = {
          width : width,
          height : height,
          plotPadding: [24,24,24,24], // top, right, bottom, left
          get plotWidth()
          {
            return layoutMeasures.width - layoutMeasures.plotPadding[1] - layoutMeasures.plotPadding[3]
          },
          get plotHeight()
          {
            return layoutMeasures.height - layoutMeasures.plotPadding[0] - layoutMeasures.plotPadding[2]
          }
        }

        // clean clear our svg
        svg.selectAll("g").remove();

        // plot container
        let container = svg
        .append("g")
        .attr("height", layoutMeasures.plotHeight)
        .attr("width", layoutMeasures.plotWidth)
        .attr("transform", `translate(${layoutMeasures.plotPadding[0]}, ${layoutMeasures.plotPadding[3]})`);

        container.append("rect")
          .attr("height", layoutMeasures.plotHeight)
          .attr("width", layoutMeasures.plotWidth)
          .style("fill", "#F7F7F7")            

        var y = d3.scaleLinear()
          .domain([0, d3.max(data, i => i.y)])
          .range([layoutMeasures.plotHeight, 0])
        var x = d3.scaleLinear()
          .domain([0, d3.max(data, i => i.x)])
          .range([0, layoutMeasures.plotWidth])
          
        var line = d3.line()
          .defined(d => !isNaN(d.x))
          .defined(d => !isNaN(d.y))
          .x(d => x(d.x))
          .y(d => y(d.y))

        var yAxis = g => g
            .attr("transform",`translate(0,0)`)
            .call(d3.axisLeft(y))
        var xAxis = g => g
            .attr("transform",`translate(0,${layoutMeasures.plotHeight})`)
            .call(d3.axisBottom(x))

        container.append("g")
            .call(yAxis);
        container.append("g")
            .call(xAxis);

        container.append("path")
          .datum(data)
          .attr("fill", "none")
          .attr("stroke", "steelblue")
          .attr("stroke-width", 1)
          .attr("d", line);
    }
    
    var svg = d3.select(sgvSelector);
    console.log(svg);
    interactive.csharp.getVariable(dataVariable)
        .then( data => buildFrameRatePlot(svg, 900, 300, data));
    });
}

In [1]:
#!html
<svg id="dataPlot1" width=900 height=300></svg>

#!js
notebookScope.plot("svg#dataPlot1", "data" );

In [1]:
#!html


In [1]:
#!html
